<a href="https://colab.research.google.com/github/Erasnilson/Trat-dados-via-PySpark/blob/main/Trat_dados_via_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamento de dados via PySpark
- Nesse exemplo utilizaremos uma base de dados da Agência Nacional de Aviação Civil (ANAC).

- Fluxo de voos no mês 06-2024



## Java e Spark

O Apache Spark é uma plataforma de processamento de dados em larga escala, projetada para ser rápida e eficiente. Ele é amplamente utilizado em várias áreas, permite a análise de grandes volumes de dados em paralelo. As tarefas podem ser distribuídas por vários nós de uma rede, o que acelera o processamento.


In [ ]:
#download do Java via colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#download do spark via colab # as vezes a versão muda, então sempre verificar e mudar a versao
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz

In [ ]:
#descompactando o spark
!tar xf spark-3.5.2-bin-hadoop3.tgz

### Instalando o findspark

A biblioteca findspark é utilizada para realizar a configuração do pyspark.
O findspark permite que você especifique o caminho de instalação do Spark manualmente dentro do seu código, sem precisar configurar variáveis de ambiente do sistema.

In [ ]:
#instalando o findspark
!pip install -q findspark


In [ ]:
#importando as bilbiotecas
import findspark
import os

In [ ]:
#configurando a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

### Iniciando uma sessão do spark em modo local

In [ ]:
!pip install pyspark==3.4.0

In [ ]:
#informando a pasta raíz do spark
findspark.init("spark-3.5.2-bin-hadoop3")

In [ ]:
#iniciando uma sessão do Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
#visualizando a versão do Spark
spark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
#iniciando o objeto SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

/content/spark-3.5.2-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Dataset [Sirios-ANAC](https://siros.anac.gov.br/siros/registros/diversos/vra/2024/)

## Base de dados da Agência Nacional de Aviação Civil (ANAC).

 - Fluxo de voos no mês 06-2024
 - Aerpoporto de Guarulhos



In [ ]:
!wget https://siros.anac.gov.br/siros/registros/diversos/vra/2024/VRA_2024_06.csv

--2024-08-12 17:31:00--  https://siros.anac.gov.br/siros/registros/diversos/vra/2024/VRA_2024_06.csv
Resolving siros.anac.gov.br (siros.anac.gov.br)... 189.84.138.178
Connecting to siros.anac.gov.br (siros.anac.gov.br)|189.84.138.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22207071 (21M) [application/octet-stream]
Saving to: ‘VRA_2024_06.csv’

VRA_2024_06.csv     100%[===================>]  21.18M   938KB/s    in 24s     

2024-08-12 17:31:25 (903 KB/s) - ‘VRA_2024_06.csv’ saved [22207071/22207071]



In [ ]:
#realizando o download da pasta
#!wget -r -np -nH --cut-dirs=4 -A csv https://siros.anac.gov.br/siros/registros/diversos/vra/2023/
#descompactando o arquivo de dados em zip
#!unzip 2019.zip

In [ ]:
data_voo = sqlContext.read.load("VRA_2024_06.csv", #pasta "2023/*"
                     format='csv',
                     header='true',
                     #lineSep='',#\r\n
                     inferSchema='true',
                     delimiter=";",
                     multiLine = 'true',
                     dateFormat = "dd/MM/yyyy",
                     timestampFormat = "dd/MM/yyyy HH:mm")

In [ ]:
# tratando a cofiguração data-hora dentro de cada pasta (por ano)
data_voo.show(5)
data_voo.printSchema()

+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+----------------------------+---------------------------+-------------------+-------------------+------------+-------------+----------+----------------+----------------+
|Sigla ICAO Empresa Aérea|       Empresa Aérea|Número Voo|Código DI|Código Tipo Linha|Modelo Equipamento|Número de Assentos|Sigla ICAO Aeroporto Origem|Descrição Aeroporto Origem|   Partida Prevista|       Partida Real|Sigla ICAO Aeroporto Destino|Descrição Aeroporto Destino|   Chegada Prevista|       Chegada Real|Situação Voo|Justificativa|Referência|Situação Partida|Situação Chegada|
+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+---


# A variável "referência"  foi classificada como string, deve ser classificada como data.

In [ ]:
# Tornando a variável referencia no mesmo padrão dos outros anos
from pyspark.sql.functions import *
#data_voov2=data_voo.withColumn('Referência', substring('Referência', 1,10))

# deixar a variavel referencia no mesmo padrao das demais
data_voov2 = data_voo.withColumn('Referência', to_date(data_voo['Referência'], 'yyyy-MM-dd'))

In [ ]:
# tratando a cofiguração data-hora dentro de cada pasta (por ano)
data_voov2.show(5)
data_voov2.printSchema()

+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+----------------------------+---------------------------+-------------------+-------------------+------------+-------------+----------+----------------+----------------+
|Sigla ICAO Empresa Aérea|       Empresa Aérea|Número Voo|Código DI|Código Tipo Linha|Modelo Equipamento|Número de Assentos|Sigla ICAO Aeroporto Origem|Descrição Aeroporto Origem|   Partida Prevista|       Partida Real|Sigla ICAO Aeroporto Destino|Descrição Aeroporto Destino|   Chegada Prevista|       Chegada Real|Situação Voo|Justificativa|Referência|Situação Partida|Situação Chegada|
+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+---

Uma das estratégias ofertadas pelo Spark, com foco em aumentar o desempenho da execução de operações sobre os dados é o método cache(), que permite guardar em memória os resultados dos dados obtidos por operações, para serem utilizados em consultas posteriores.

In [ ]:
spark

In [ ]:
#criando um cache dos dados
df_train = data_voov2.cache()

## Organizando a base de dados

In [ ]:
# deletar coluna que possui null
# eliminando colunas que sem "data de partidas"/"chegada do voo" (Referência = data prevista sem horas)
df_train = df_train.dropna(subset=('Partida Real','Chegada Real','Referência'))
df_train.show(5)

+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+----------------------------+---------------------------+-------------------+-------------------+------------+-------------+----------+----------------+----------------+
|Sigla ICAO Empresa Aérea|       Empresa Aérea|Número Voo|Código DI|Código Tipo Linha|Modelo Equipamento|Número de Assentos|Sigla ICAO Aeroporto Origem|Descrição Aeroporto Origem|   Partida Prevista|       Partida Real|Sigla ICAO Aeroporto Destino|Descrição Aeroporto Destino|   Chegada Prevista|       Chegada Real|Situação Voo|Justificativa|Referência|Situação Partida|Situação Chegada|
+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+---

# Criando novas variáveis a partir da data de Referência

In [ ]:
# subdividir uma variavel por carachter
from pyspark.sql.types import IntegerType

df_train=df_train.withColumn('Refyear', substring('Referência', 1,4).cast(IntegerType()))\
.withColumn('Refmonth', substring('Referência', 6,2).cast(IntegerType()))\
.withColumn('Refday', substring('Referência', 9,2).cast(IntegerType()))

df_train.show(5)
df_train.printSchema()
# 4 digitos (incluíndo 1) = ano  ex: 2024-06-01
# 2 digitos (incluíndo 6) = mês
# 2 digitos (incluíndo 9) = mês

+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+----------------------------+---------------------------+-------------------+-------------------+------------+-------------+----------+----------------+----------------+----+-----+---+-------+--------+------+
|Sigla ICAO Empresa Aérea|       Empresa Aérea|Número Voo|Código DI|Código Tipo Linha|Modelo Equipamento|Número de Assentos|Sigla ICAO Aeroporto Origem|Descrição Aeroporto Origem|   Partida Prevista|       Partida Real|Sigla ICAO Aeroporto Destino|Descrição Aeroporto Destino|   Chegada Prevista|       Chegada Real|Situação Voo|Justificativa|Referência|Situação Partida|Situação Chegada|year|month|day|Refyear|Refmonth|Refday|
+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+--------------------

In [ ]:
from pyspark.sql.functions import col
# Filtrar a base de dados que será utilizada (considerando uma pré-análise)
df_train.filter( (col('Situação Voo') == "REALIZADO") & (col('Sigla ICAO Aeroporto Origem') == "SBGR") & (col('Refyear')==2024) ).show(5)
# SBGR - Guarulhos

+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+---------------------------+--------------------------+-------------------+-------------------+----------------------------+---------------------------+-------------------+-------------------+------------+-------------+----------+----------------+----------------+----+-----+---+-------+--------+------+
|Sigla ICAO Empresa Aérea|       Empresa Aérea|Número Voo|Código DI|Código Tipo Linha|Modelo Equipamento|Número de Assentos|Sigla ICAO Aeroporto Origem|Descrição Aeroporto Origem|   Partida Prevista|       Partida Real|Sigla ICAO Aeroporto Destino|Descrição Aeroporto Destino|   Chegada Prevista|       Chegada Real|Situação Voo|Justificativa|Referência|Situação Partida|Situação Chegada|year|month|day|Refyear|Refmonth|Refday|
+------------------------+--------------------+----------+---------+-----------------+------------------+------------------+--------------------

# Filtros em pyspark/pandas


In [ ]:
# Demais filtros
#df_train.filter( (col('Situação Voo') == "REALIZADO") & (col('Sigla ICAO Aeroporto Origem') == "SBGR") & (col('year')==2024) ).toPandas().head(5)
#df_train.groupBy('year','month').agg({"Referência": "count"}).sort('year','month').toPandas()
#df_train.groupBy('Situação Chegada',"Empresa Aérea","DiffInHoursR").count().show()
#df_train.groupBy('Situação Chegada',"Empresa Aérea","Data_Ref").agg({"horas_voo": "avg"}).sort("Empresa Aérea").show()
#'Sigla ICAO Aeroporto Destino'Referência .sort('year','month')
#.sort("Situação Voo", "Número de Assentos")
#.agg({"salary": "avg", "age": "max"}).show()

## Groupby

In [ ]:
from pandas.core.indexes.api import safe_sort_index
from numpy import select
from pandas.core.groupby import groupby

# Voos por dia em Guarulhos
df_train.groupBy("Referência").agg({"Referência": "count"}).sort('Referência').show(5)

# Renomear as variáveis
from pyspark.sql.functions import *
#df_train.withColumnRenamed("Referência","Data").withColumnRenamed("count(Referência)","n")

+----------+-----------------+
|Referência|count(Referência)|
+----------+-----------------+
|2024-06-01|             1886|
|2024-06-02|             2365|
|2024-06-03|             2570|
|2024-06-04|             2563|
|2024-06-05|             2555|
+----------+-----------------+
only showing top 5 rows



## Pandas

In [ ]:
# Converter o DataFrame PySpark para um DataFrame Pandas
pd_df = df_train.toPandas()

In [ ]:
!pip install unidecode
import unidecode

pd_df.columns = (
    pd_df.columns
    .map(unidecode.unidecode)  # Remove a acentuação de todas as letras
    .str.replace('[^A-Za-z0-9_]', '', regex=True)  # Remove caracteres especiais, mantendo letras e números
)
pd_df.columns

Index(['SiglaICAOEmpresaAerea', 'EmpresaAerea', 'NumeroVoo', 'CodigoDI',
       'CodigoTipoLinha', 'ModeloEquipamento', 'NumerodeAssentos',
       'SiglaICAOAeroportoOrigem', 'DescricaoAeroportoOrigem',
       'PartidaPrevista', 'PartidaReal', 'SiglaICAOAeroportoDestino',
       'DescricaoAeroportoDestino', 'ChegadaPrevista', 'ChegadaReal',
       'SituacaoVoo', 'Justificativa', 'Referencia', 'SituacaoPartida',
       'SituacaoChegada', 'year', 'month', 'day', 'Refyear', 'Refmonth',
       'Refday'],
      dtype='object')

# Análise Exploratória

In [ ]:
# Filtrar os dados com as condições desejadas e mostrar as 5 primeiras linhas
filter_SBGR = pd_df[(pd_df['SituacaoVoo'] == "REALIZADO") &
                    (pd_df['SiglaICAOAeroportoOrigem'] == "SBGR") &
                    (pd_df['year'] == 2024)]

filtered_df = filter_SBGR[['Referencia', 'PartidaReal', 'EmpresaAerea', 'ModeloEquipamento','SituacaoChegada']]
filtered_df.head(3)

,Referencia,PartidaReal,EmpresaAerea,ModeloEquipamento,SituacaoChegada
2,2024-06-01,2024-06-01 23:08:00,"AMERICAN AIRLINES, INC.",B772,Antecipado
4,2024-06-01,2024-06-01 21:18:00,"AMERICAN AIRLINES, INC.",B77W,Antecipado
6,2024-06-01,2024-06-01 22:16:00,"AMERICAN AIRLINES, INC.",B789,Antecipado


In [ ]:
#Alterando tipo de coluna: mes e hora da partida real
filtered_df['Months'] = filtered_df['PartidaReal'].dt.month
filtered_df['Hour'] = filtered_df['PartidaReal'].dt.hour
filtered_df['DataPart'] = filtered_df['PartidaReal'].dt.date
filtered_df.head(3)

<ipython-input-94-64c3350109f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Months'] = filtered_df['PartidaReal'].dt.month
<ipython-input-94-64c3350109f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Hour'] = filtered_df['PartidaReal'].dt.hour
<ipython-input-94-64c3350109f8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,Referencia,PartidaReal,EmpresaAerea,ModeloEquipamento,SituacaoChegada,Months,Hour,DataPart
2,2024-06-01,2024-06-01 23:08:00,"AMERICAN AIRLINES, INC.",B772,Antecipado,6,23,2024-06-01
4,2024-06-01,2024-06-01 21:18:00,"AMERICAN AIRLINES, INC.",B77W,Antecipado,6,21,2024-06-01
6,2024-06-01,2024-06-01 22:16:00,"AMERICAN AIRLINES, INC.",B789,Antecipado,6,22,2024-06-01


## Resumindo os dados com Tabelas de Frequência

In [ ]:
frequencia = filtered_df['EmpresaAerea'].value_counts()
frequencia_relativa = filtered_df['EmpresaAerea'].value_counts(normalize=True) * 100

tb_frequencia = pd.DataFrame({'Frequência': frequencia, 'Frequência Relativa (%)': frequencia_relativa})\
                    .rename_axis('Empresa Aérea')\
                        .reset_index()

tb_frequencia['Frequência Acumulada'] = tb_frequencia['Frequência'].cumsum()
tb_frequencia['Frequência Relativa Acumulada (%)'] = tb_frequencia['Frequência Acumulada'] / tb_frequencia['Frequência'].sum() * 100
tb_frequencia

,Empresa Aérea,Frequência,Frequência Relativa (%),Frequência Acumulada,Frequência Relativa Acumulada (%)
0,TAM LINHAS AÉREAS S.A.,5314,47.676296,5314,47.676296
1,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,2473,22.187332,7787,69.863628
2,AZUL LINHAS AÉREAS BRASILEIRAS S/A,745,6.684012,8532,76.547640
3,PASSAREDO TRANSPORTES AÉREOS S.A.,264,2.368563,8796,78.916203
4,LATAM AIRLINES GROUP (EX - LAN AIRLINES S/A),211,1.893056,9007,80.809259
5,SIDERAL LINHAS AÉREAS LTDA.,193,1.731563,9200,82.540822
6,COMPAÑIA PANAMEÑA DE AVIACION S.A. (COPA AIRLI...,160,1.435493,9360,83.976314
7,"AMERICAN AIRLINES, INC.",119,1.067648,9479,85.043962
8,"UNITED AIRLINES, INC",117,1.049704,9596,86.093666
9,AEROLINEAS ARGENTINAS S/A,112,1.004845,9708,87.098511


## Totais por grupo

In [ ]:
tot_voos = pd.crosstab(index=filtered_df['Hour'],
                        columns=filtered_df['SituacaoChegada'],
                        values=filtered_df['EmpresaAerea'],
                        aggfunc='count')  # sum

tot_voos.reset_index(drop=False, inplace=True)
tot_voos.columns.name = None
tot_voos

,Hour,Antecipado,Atraso 120-240,Atraso 30-60,Atraso 60-120,Atraso > 240,Pontual
0,0,49.0,2.0,12.0,4.0,NaN,28.0
1,1,164.0,4.0,5.0,3.0,1.0,37.0
2,2,33.0,3.0,6.0,NaN,3.0,30.0
3,3,61.0,1.0,44.0,2.0,4.0,61.0
4,4,141.0,7.0,25.0,12.0,2.0,59.0
5,5,159.0,NaN,1.0,2.0,NaN,29.0
6,6,148.0,2.0,4.0,1.0,1.0,31.0
7,7,494.0,3.0,8.0,1.0,1.0,280.0
8,8,410.0,1.0,46.0,6.0,1.0,390.0
9,9,510.0,3.0,35.0,16.0,4.0,398.0


# Quantitativo de voos, por hora e data, no aeroporto de Guarulhos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

atraso_por_hora = pd.crosstab(index=filtered_df['DataPart'],
                              columns=filtered_df['Hour'],
                              values=filtered_df['EmpresaAerea'],
                              aggfunc='count').fillna(0).astype('int')

In [ ]:
atraso_por_hora.style.background_gradient(cmap='rocket').set_properties(**{'text-align': 'center'})

Hour,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
DataPart,,,,,,,,,,,,,,,,,,,,,,,,
2024-06-01,0,6,2,6,8,7,6,22,29,29,10,10,22,6,12,12,13,19,21,6,9,18,20,22
2024-06-02,2,8,4,1,5,5,4,25,29,31,9,5,22,16,18,18,18,26,18,4,13,22,29,28
2024-06-03,4,6,3,2,5,6,6,25,32,30,9,5,21,13,17,19,13,29,19,5,9,22,27,29
2024-06-04,5,9,3,9,9,8,3,29,28,35,8,4,22,14,21,17,16,21,17,7,8,23,28,27
2024-06-05,8,6,3,6,12,5,9,28,32,33,7,5,17,18,18,21,14,27,18,5,10,21,28,29
2024-06-06,3,5,3,8,8,8,7,29,25,37,8,3,21,13,16,21,13,26,20,7,9,24,33,25
2024-06-07,4,7,4,6,11,7,7,22,28,30,18,11,19,14,19,20,16,28,14,8,9,21,31,28
2024-06-08,3,5,4,9,9,6,8,28,28,30,10,10,20,11,18,16,16,24,19,6,9,16,27,23
2024-06-09,1,7,1,4,2,7,5,11,34,31,13,8,23,13,21,13,18,27,19,6,13,19,33,23


# Demais códigos

In [ ]:
#get info about the dataset:
print(filtered_df.info())
print("-"*25)
# to know more about the shape of the dataset:
print("shape of the dataset -->>",np.shape(filtered_df))

<class 'pandas.core.frame.DataFrame'>
Index: 11146 entries, 2 to 75195
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   SiglaICAOEmpresaAerea      11146 non-null  object        
 1   EmpresaAerea               11146 non-null  object        
 2   NumeroVoo                  11146 non-null  object        
 3   CodigoDI                   11146 non-null  object        
 4   CodigoTipoLinha            11146 non-null  object        
 5   ModeloEquipamento          11146 non-null  object        
 6   NumerodeAssentos           11146 non-null  int32         
 7   SiglaICAOAeroportoOrigem   11146 non-null  object        
 8   DescricaoAeroportoOrigem   11146 non-null  object        
 9   PartidaPrevista            10919 non-null  datetime64[ns]
 10  PartidaReal                11146 non-null  datetime64[ns]
 11  SiglaICAOAeroportoDestino  11146 non-null  object        
 12  Descricao

In [ ]:
#check if there null values in data:
filtered_df.isna().mean()


,0
SiglaICAOEmpresaAerea,0.000000
EmpresaAerea,0.000000
NumeroVoo,0.000000
CodigoDI,0.000000
CodigoTipoLinha,0.000000
ModeloEquipamento,0.000000
NumerodeAssentos,0.000000
SiglaICAOAeroportoOrigem,0.000000
DescricaoAeroportoOrigem,0.000000
PartidaPrevista,0.020366


# Quantitativo de cada item em cada coluna:

In [ ]:
# Especificar as colunas que você deseja iterar
variaveis = ['EmpresaAerea', 'ModeloEquipamento', 'DescricaoAeroportoOrigem']

# Loop para contar itens nas colunas especificadas
for col in variaveis:
    print(f"Counts of items in {col} -->> {filtered_df[col].value_counts()}") #\n
    print("-" * 25)

Counts of items in EmpresaAerea -->> EmpresaAerea
TAM LINHAS AÉREAS S.A.                                       5314
GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)          2473
AZUL LINHAS AÉREAS BRASILEIRAS S/A                            745
PASSAREDO TRANSPORTES AÉREOS S.A.                             264
LATAM AIRLINES GROUP (EX - LAN AIRLINES S/A)                  211
SIDERAL LINHAS AÉREAS LTDA.                                   193
COMPAÑIA PANAMEÑA DE AVIACION S.A. (COPA AIRLINES)            160
AMERICAN AIRLINES, INC.                                       119
UNITED AIRLINES, INC                                          117
AEROLINEAS ARGENTINAS S/A                                     112
TAP - TRANSPORTES AÉREOS PORTUGUESES S/A                       92
AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA                 90
DELTA AIR LINES INC.                                           90
TURKISH AIRLINES INC                                           85
QATAR AIRWAYS GROUP       